<a href="https://colab.research.google.com/github/vvegag/RECOMENDATION_SYSTEMS/blob/main/ALS_recomendation_SPARK_EXAMPLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [54]:
#wget -q "http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tg"

In [ ]:
#tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [2]:
#pip install -q findspark

In [3]:
 #pip install pyspark

In [10]:
#import pandas as pd

In [26]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.getOrCreate()

from pyspark.ml.recommendation import ALS, ALSModel  ## Import ALSModel along with ALS

spark = SparkSession.builder.getOrCreate()

In [27]:
import tempfile
# Create a temporary directory using tempfile
temp_path2 = tempfile.mkdtemp()

In [36]:
temp_path = "/content/Test2"

In [37]:
temp_path

'/content/Test2'

In [29]:
df = spark.createDataFrame(
    [(0, 0, 4.0), (0, 1, 2.0), (1, 1, 3.0), (1, 2, 4.0), (2, 1, 1.0), (2, 2, 5.0)],
    ["user", "item", "rating"])
als = ALS(rank=10, seed=0)
als.setMaxIter(5)

als.getMaxIter()

als.setRegParam(0.1)

als.getRegParam()

als.clear(als.regParam)
model = als.fit(df)
model.getBlockSize()

model.getUserCol()

model.setUserCol("user")

model.getItemCol()

model.setPredictionCol("newPrediction")

model.rank

model.userFactors.orderBy("id").collect()

test = spark.createDataFrame([(0, 2), (1, 0), (2, 0)], ["user", "item"])
predictions = sorted(model.transform(test).collect(), key=lambda r: r[0])
predictions[0]

predictions[1]

predictions[2]

user_recs = model.recommendForAllUsers(3)
user_recs.where(user_recs.user == 0)        .select("recommendations.item", "recommendations.rating").collect()

item_recs = model.recommendForAllItems(3)
item_recs.where(item_recs.item == 2)        .select("recommendations.user", "recommendations.rating").collect()

user_subset = df.where(df.user == 2)
user_subset_recs = model.recommendForUserSubset(user_subset, 3)
user_subset_recs.select("recommendations.item", "recommendations.rating").first()

item_subset = df.where(df.item == 0)
item_subset_recs = model.recommendForItemSubset(item_subset, 3)
item_subset_recs.select("recommendations.user", "recommendations.rating").first()




Row(user=[0, 1, 2], rating=[3.9102137088775635, 3.473569869995117, -0.899198055267334])

In [38]:
predictions[2]


Row(user=2, item=0, newPrediction=-0.899198055267334)

In [39]:
# Now you can use temp_path to save and load the model
als_path = temp_path + "/als"
als.save(als_path)
als2 = ALS.load(als_path)
als.getMaxIter()

model_path = temp_path + "/als_model"
model.save(model_path)


In [45]:
model_path


'/content/Test2/als_model'

In [40]:
model2 = ALSModel.load(model_path)

model.rank == model2.rank

sorted(model.userFactors.collect()) == sorted(model2.userFactors.collect())

sorted(model.itemFactors.collect()) == sorted(model2.itemFactors.collect())

model.transform(test).take(1) == model2.transform(test).take(1)

True

In [53]:
#Testando o modelo 2
predictions = sorted(model2.transform(test).collect(), key=lambda r: r[0])
predictions[0]

Row(user=0, item=2, newPrediction=0.6929104924201965)